https://womantalks.ru/forum/11-sekretyi-krasotyi/

In [1]:
# pip install bs

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm

In [1]:
df = pd.DataFrame(columns=['text'])
df

,text


In [51]:
url = 'https://womantalks.ru/search/?tags=%D0%BF%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D1%8B+%D1%81+%D0%B2%D0%BE%D0%BB%D0%BE%D1%81%D0%B0%D0%BC%D0%B8'

In [52]:
response = requests.get(url)
print(response)

<Response [200]>


In [53]:
html = response.text

In [54]:
# bs = BeautifulSoup(response.text,"lxml")
# print(bs)

In [55]:
import re

In [60]:
# soup = BeautifulSoup(html, features="html.parser")

soup = BeautifulSoup(urlopen(str('https://womantalks.ru/search/?tags=%D0%BF%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D1%8B+%D1%81+%D0%B2%D0%BE%D0%BB%D0%BE%D1%81%D0%B0%D0%BC%D0%B8'.encode('utf-8'))[2:-1]), features="html.parser")

# user
tds = soup.find_all('strong', 'itemprop'=="name")
for td in tds:
    if 'Изменено' not in td.text:
        print(td.text)


<strong>ИЛИ</strong>
ИЛИ
<strong>И</strong>
И


In [67]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import json

def get_html(url):
    response = requests.get(url)
    return response.text

def get_all_links(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_large ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            links.append(q['href'])
    return links

# def get_all_links_on_page(html, link):
#     soup = BeautifulSoup(html, features="html.parser")
#     tds = soup.find_all('tr')
#     for td in tds:
#         i = td.find('td', {'class':'td-block'})
#         if i != None:
#             return(i.find_all('a')[-1].text)
    
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    if len(links) == 0:
        return [link]
    return sorted(list(set(links)))

def get_all_links_one_theme(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            links.append(q['href'])
    return links

def get_page_data(html):
    soup = BeautifulSoup(urlopen(str(html.encode('utf-8'))[2:-1]), features="html.parser")

    # user
    tds = soup.find_all('strong', 'itemprop'=="name")
    users = []
    for td in tds:
        if 'Изменено' not in td.text:
            users.append(td.text)
#     print(users)


    # time
    tds = soup.find_all('p', {'class':"ipsType_reset"})
    times = []
    for td in tds:
        if 'Опубликовано' in td.text:
            times.append(td.text)
#     print(times)


    # text
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    tex = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find_all('p')
            tex.append(p[-1].text)
        else:
            tex.append(td.text)
#     print(tex)
    

    # quote
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    temp = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find('div', {'class':"ipsQuote_contents"})
            if p is not None:
                temp.append(p.text)
        else:
            temp.append('')
#     print(temp)

    data = ({'user': users,                    
             'time': times,      
             'text': tex,            
             'quote': temp,          
             'topic': ['Beauty'] * len(users)
            })          
    return users, times, tex, temp, ['Beauty'] * len(users)#data

def page_content(all_links, table):
    for link in tqdm(all_links):
        links = get_links(get_html(link), link)
        for l in links:
            all_links_one_theme = get_all_links_one_theme(get_html(l))
            for t in tqdm(all_links_one_theme):
                page = get_links(get_html(t), t)
                for p in page:
                    if 'search' not in str(p):
                        print(p)
                        us, ti, te, tem, l = get_page_data(p)
                        for i in range(len(us)):
                            if i > 5:
                                break
                            users.append(us[i])
                            times.append(ti[i])
                            tex.append(te[i])
                            temp.append(tem[i])
                            labels.append(l[i])

table = pd.DataFrame(columns={'users', 'times', 'text', 'quote', 'topic'})
users = []
times = []
tex = []
temp = []
labels = []
    
url = 'https://womantalks.ru/forum/11-sekretyi-krasotyi/'

all_links = get_all_links(get_html(url))
page_content(all_links, table)

table.loc[:, 'users'] = users
table.loc[:, 'times'] = times
table.loc[:, 'text'] = tex
table.loc[:, 'quote'] = temp
table.loc[:, 'topic'] = labels

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

https://womantalks.ru/topic/283-kak-sdelat-kudri-utyuzhkom/?page=1
https://womantalks.ru/topic/283-kak-sdelat-kudri-utyuzhkom/?page=2
https://womantalks.ru/topic/7967-volosyi-vyipadayut/
https://womantalks.ru/topic/1689-perhot-i-shampun/?page=1
https://womantalks.ru/topic/1689-perhot-i-shampun/?page=2
https://womantalks.ru/topic/1689-perhot-i-shampun/?page=3
https://womantalks.ru/topic/1689-perhot-i-shampun/?page=4
https://womantalks.ru/topic/1689-perhot-i-shampun/?page=5
https://womantalks.ru/topic/5413-chto-delat-esli-ochen-suhie-volosyi/?page=1
https://womantalks.ru/topic/5413-chto-delat-esli-ochen-suhie-volosyi/?page=2
https://womantalks.ru/topic/5144-ne-mogu-promyit-volosyi/?page=1
https://womantalks.ru/topic/5144-ne-mogu-promyit-volosyi/?page=2
https://womantalks.ru/topic/8254-dlinnyie-volosyi/
https://womantalks.ru/topic/8423-podderzhanie-tsveta-volos/
https://womantalks.ru/topic/8595-hochu-obuchitsya-izgotovleniyu-masok-dlya-volos/
https://womantalks.ru/topic/6549-lyubimyie-sre

  0%|          | 0/25 [00:00<?, ?it/s]

https://womantalks.ru/topic/378-konditsioner-dlya-volos/?page=1
https://womantalks.ru/topic/378-konditsioner-dlya-volos/?page=2
https://womantalks.ru/topic/378-konditsioner-dlya-volos/?page=3
https://womantalks.ru/topic/8112-uhod-za-blondom/
https://womantalks.ru/topic/5255-sredstva-protiv-vyipadeniya-volos/?page=1
https://womantalks.ru/topic/5255-sredstva-protiv-vyipadeniya-volos/?page=2
https://womantalks.ru/topic/7951-dlinnyie-volosyi/
https://womantalks.ru/topic/4480-kak-vyi-spasaete-volosyi-osenyu-i-vesnoy/?page=1
https://womantalks.ru/topic/4480-kak-vyi-spasaete-volosyi-osenyu-i-vesnoy/?page=2
https://womantalks.ru/topic/6464-shampun-dlya-neposlushnyih-volos/
https://womantalks.ru/topic/660-masla-dlya-volos/?page=1
https://womantalks.ru/topic/660-masla-dlya-volos/?page=2
https://womantalks.ru/topic/4226-posovetuyte-shapnun-dlya-ukrepleniya-volos/?page=1
https://womantalks.ru/topic/4226-posovetuyte-shapnun-dlya-ukrepleniya-volos/?page=2
https://womantalks.ru/topic/5216-maski-dlya-

...

  0%|          | 0/25 [00:00<?, ?it/s]

https://womantalks.ru/topic/4995-kak-ubrat-meshki-pod-glazami/?page=1
https://womantalks.ru/topic/4995-kak-ubrat-meshki-pod-glazami/?page=2
https://womantalks.ru/topic/4995-kak-ubrat-meshki-pod-glazami/?page=3
https://womantalks.ru/topic/31-kak-izbavitsya-ot-akne-pryishhey/?page=1
https://womantalks.ru/topic/31-kak-izbavitsya-ot-akne-pryishhey/?page=2
https://womantalks.ru/topic/31-kak-izbavitsya-ot-akne-pryishhey/?page=3
https://womantalks.ru/topic/31-kak-izbavitsya-ot-akne-pryishhey/?page=4
https://womantalks.ru/topic/31-kak-izbavitsya-ot-akne-pryishhey/?page=5
https://womantalks.ru/topic/31-kak-izbavitsya-ot-akne-pryishhey/?page=6
https://womantalks.ru/topic/823-kak-izbavitsya-ot-suhosti-litsa/?page=1
https://womantalks.ru/topic/823-kak-izbavitsya-ot-suhosti-litsa/?page=2
https://womantalks.ru/topic/823-kak-izbavitsya-ot-suhosti-litsa/?page=3
https://womantalks.ru/topic/823-kak-izbavitsya-ot-suhosti-litsa/?page=4
https://womantalks.ru/topic/823-kak-izbavitsya-ot-suhosti-litsa/?page=

  0%|          | 0/26 [00:00<?, ?it/s]

https://womantalks.ru/topic/7303-feys-fitnes/
https://womantalks.ru/topic/953-kak-izbavitsya-ot-pryishhey/?page=1
https://womantalks.ru/topic/953-kak-izbavitsya-ot-pryishhey/?page=2
https://womantalks.ru/topic/953-kak-izbavitsya-ot-pryishhey/?page=3
https://womantalks.ru/topic/953-kak-izbavitsya-ot-pryishhey/?page=4


IndexError: list index out of range

In [100]:
len(users)

793

In [69]:
len(times)

793

In [70]:
len(tex)

793

In [71]:
len(temp)

793

In [72]:
len(labels)

793

In [81]:
table.loc[:, 'users'] = users
table.loc[:, 'times'] = times
table.loc[:, 'text'] = tex
table.loc[:, 'quote'] = temp
table.loc[:, 'topic'] = labels

In [82]:
table[['users', 'times', 'text', 'quote', 'topic']]

,users,times,text,quote,topic
0,\nwomantalks.ru,\nОпубликовано: 5 сен 2015\n,\n\r\n\tКудри издавна считались особым элемент...,,Beauty
1,\nКиана,\nОпубликовано: 19 май 2016\n,\n\n\tВсе время делаю себе кудри именно утюжко...,,Beauty
2,\nelenavolk,\nОпубликовано: 22 май 2016\n,"\n\n\tкрасиво то как, и я так хочу)\n\n",,Beauty
3,\nlena.a,\nОпубликовано: 24 май 2016\n,"\n\n\tкрасиво, но волосы портятся будь здоров)...",,Beauty
4,\nЛасковая,\nОпубликовано: 27 авг 2016\n,"\n\n\tЭто точно, от плоек и утюжков волосы сил...",,Beauty
...,...,...,...,...,...
788,\nfarovskayaoksana,\nОпубликовано: 27 май 2018\n,\n\n\tМне очень здорово серия серацин помогает...,,Beauty
789,\nГалина,\nОпубликовано: 17 июл 2019\n,"\n\n\tПро ПП лекции читать не собираюсь, но эт...",,Beauty
790,по моему при прыщах скрабить лицо не стоит. Пр...,\nОпубликовано: 16 дек 2019\n,\n\nпо моему при прыщах скрабить лицо не стоит...,,Beauty
791,\nnadin-er,\nОпубликовано: 25 май 2020\n,\n\n\tу дочки на лице угри пошли. Хоть и следи...,,Beauty


In [93]:
table

,users,times,text,topic,quote
0,\nwomantalks.ru,\nОпубликовано: 5 сен 2015\n,\n\r\n\tКудри издавна считались особым элемент...,Beauty,
1,\nКиана,\nОпубликовано: 19 май 2016\n,\n\n\tВсе время делаю себе кудри именно утюжко...,Beauty,
2,\nelenavolk,\nОпубликовано: 22 май 2016\n,"\n\n\tкрасиво то как, и я так хочу)\n\n",Beauty,
3,\nlena.a,\nОпубликовано: 24 май 2016\n,"\n\n\tкрасиво, но волосы портятся будь здоров)...",Beauty,
4,\nЛасковая,\nОпубликовано: 27 авг 2016\n,"\n\n\tЭто точно, от плоек и утюжков волосы сил...",Beauty,
...,...,...,...,...,...
787,\nВалерия Соломонова,\nОпубликовано: 17 апр 2018\n,"\n\n\tтут спор про ретасол вышел, тоже хочу св...",Beauty,
788,\nfarovskayaoksana,\nОпубликовано: 27 май 2018\n,\n\n\tМне очень здорово серия серацин помогает...,Beauty,
789,\nГалина,\nОпубликовано: 17 июл 2019\n,"\n\n\tПро ПП лекции читать не собираюсь, но эт...",Beauty,
790,\nnadin-er,\nОпубликовано: 25 май 2020\n,\n\n\tу дочки на лице угри пошли. Хоть и следи...,Beauty,


In [94]:
table.to_csv('Beauty_woman_talks_Copy1.csv', index=False)

In [3]:
D = pd.read_csv('Beauty_woman_talks_Copy1.csv')
D

,users,times,text,topic,quote
0,\nwomantalks.ru,\nОпубликовано: 5 сен 2015\n,\n\r\n\tКудри издавна считались особым элемент...,Beauty,NaN
1,\nКиана,\nОпубликовано: 19 май 2016\n,\n\n\tВсе время делаю себе кудри именно утюжко...,Beauty,NaN
2,\nelenavolk,\nОпубликовано: 22 май 2016\n,"\n\n\tкрасиво то как, и я так хочу)\n\n",Beauty,NaN
3,\nlena.a,\nОпубликовано: 24 май 2016\n,"\n\n\tкрасиво, но волосы портятся будь здоров)...",Beauty,NaN
4,\nЛасковая,\nОпубликовано: 27 авг 2016\n,"\n\n\tЭто точно, от плоек и утюжков волосы сил...",Beauty,NaN
...,...,...,...,...,...
787,\nВалерия Соломонова,\nОпубликовано: 17 апр 2018\n,"\n\n\tтут спор про ретасол вышел, тоже хочу св...",Beauty,NaN
788,\nfarovskayaoksana,\nОпубликовано: 27 май 2018\n,\n\n\tМне очень здорово серия серацин помогает...,Beauty,NaN
789,\nГалина,\nОпубликовано: 17 июл 2019\n,"\n\n\tПро ПП лекции читать не собираюсь, но эт...",Beauty,NaN
790,\nnadin-er,\nОпубликовано: 25 май 2020\n,\n\n\tу дочки на лице угри пошли. Хоть и следи...,Beauty,NaN


In [13]:
import re

In [19]:
D.iloc[0]['text']

'\n\r\n\tКудри издавна считались особым элементом прически. Они вселяют невинность в образ, а также нежность и беззаботность. В некоторых случаях обольщения мужских сердец и глаз, без этого компонента попросту не обойтись. Однако многие девушки сталкиваются с вопросом: как их сделать правильно? Иногда достигнуть нужного результата не позволяет жесткость волоса. Профессиональный женский блог\xa0«Мюсли» сделал подборку самых эффективных методов.\xa0\r\n\tМетоды завивки волос в кудри\xa0\r\n\n\r\n\tЗа всю историю, женская половина человечества накопила богатый опыт в завивке волос, создавая самые разные и причудливые рецепты. Некоторые из них канули в лета, а другие по-прежнему применяются и с большим успехом! Главное – следовать инструкциям, о которых подробно пишется на Mjusli.ru. Тогда и результат будет вполне эффектным. Самые популярные методы:\xa0\r\n\n\r\n\t\tплойка – самый распространенный способ. Выбирайте диаметр плойки по принципу, чем меньше длина волоса, тем уже диаметр инстру

In [20]:
re.sub(r'[\n\t\r\xa0]', '', D.iloc[0]['text'])

'Кудри издавна считались особым элементом прически. Они вселяют невинность в образ, а также нежность и беззаботность. В некоторых случаях обольщения мужских сердец и глаз, без этого компонента попросту не обойтись. Однако многие девушки сталкиваются с вопросом: как их сделать правильно? Иногда достигнуть нужного результата не позволяет жесткость волоса. Профессиональный женский блог«Мюсли» сделал подборку самых эффективных методов.Методы завивки волос в кудриЗа всю историю, женская половина человечества накопила богатый опыт в завивке волос, создавая самые разные и причудливые рецепты. Некоторые из них канули в лета, а другие по-прежнему применяются и с большим успехом! Главное – следовать инструкциям, о которых подробно пишется на Mjusli.ru. Тогда и результат будет вполне эффектным. Самые популярные методы:плойка – самый распространенный способ. Выбирайте диаметр плойки по принципу, чем меньше длина волоса, тем уже диаметр инструмента. Лучший вариант – с турмалиновым покрытием;бигуди.

In [26]:
D.loc[:, 'text'] = D.text.apply(lambda x: re.sub(r'[\n\t\r\xa0]', '', str(x)))

In [30]:
D.loc[:, 'quote'] = D.quote.apply(lambda x: re.sub(r'[\n\t\r\xa0]', '', str(x)))

In [28]:
D.loc[:, 'users'] = D.users.apply(lambda x: re.sub(r'[\n\t\r\xa0]', '', str(x)))

In [32]:
D.loc[:, 'times'] = D.times.apply(lambda x: re.sub(r'[\n\t\r\xa0]', '', str(x)))

In [34]:
D.loc[:, 'times'] = D.times.apply(lambda x: re.sub(r'Опубликовано: ', '', str(x)))

In [35]:
D

,users,times,text,topic,quote
0,womantalks.ru,5 сен 2015,Кудри издавна считались особым элементом приче...,Beauty,nan
1,Киана,19 май 2016,Все время делаю себе кудри именно утюжком полу...,Beauty,nan
2,elenavolk,22 май 2016,"красиво то как, и я так хочу)",Beauty,nan
3,lena.a,24 май 2016,"красиво, но волосы портятся будь здоров)",Beauty,nan
4,Ласковая,27 авг 2016,"Это точно, от плоек и утюжков волосы сильнее п...",Beauty,nan
...,...,...,...,...,...
787,Валерия Соломонова,17 апр 2018,"тут спор про ретасол вышел, тоже хочу свое мне...",Beauty,nan
788,farovskayaoksana,27 май 2018,Мне очень здорово серия серацин помогает ) поп...,Beauty,nan
789,Галина,17 июл 2019,"Про ПП лекции читать не собираюсь, но это прав...",Beauty,nan
790,nadin-er,25 май 2020,у дочки на лице угри пошли. Хоть и следит за к...,Beauty,nan


In [36]:
D.to_csv('Beauty.csv', index=False)

In [37]:
D = pd.read_csv('Beauty.csv')
D

,users,times,text,topic,quote
0,womantalks.ru,5 сен 2015,Кудри издавна считались особым элементом приче...,Beauty,NaN
1,Киана,19 май 2016,Все время делаю себе кудри именно утюжком полу...,Beauty,NaN
2,elenavolk,22 май 2016,"красиво то как, и я так хочу)",Beauty,NaN
3,lena.a,24 май 2016,"красиво, но волосы портятся будь здоров)",Beauty,NaN
4,Ласковая,27 авг 2016,"Это точно, от плоек и утюжков волосы сильнее п...",Beauty,NaN
...,...,...,...,...,...
787,Валерия Соломонова,17 апр 2018,"тут спор про ретасол вышел, тоже хочу свое мне...",Beauty,NaN
788,farovskayaoksana,27 май 2018,Мне очень здорово серия серацин помогает ) поп...,Beauty,NaN
789,Галина,17 июл 2019,"Про ПП лекции читать не собираюсь, но это прав...",Beauty,NaN
790,nadin-er,25 май 2020,у дочки на лице угри пошли. Хоть и следит за к...,Beauty,NaN


In [39]:
import random
import numpy as np
ind = np.random.choice(len(D), 50, False)
ind

array([498, 427, 406,  73, 433, 605, 630, 483, 649,   1,  54, 530, 245,
       471, 514, 152, 552, 202, 428, 567, 415, 233, 100, 174, 258, 345,
       379, 475, 219, 116, 166,  26, 350, 262, 476, 782, 276, 439, 652,
       337, 293, 616, 507, 487, 147, 291,  63, 590, 741,  77])

In [40]:
X = []
for i in ind:
    X.append(D['text'][i])

In [45]:
count = 0 # счетчик хороших записей

In [137]:
X[50]

IndexError: list index out of range

In [136]:
count+=1
print(count)

42


In [138]:
42/(50)

0.84